In [91]:
import numpy as np
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup
import datetime

pd.options.mode.chained_assignment =  None

In [26]:
def replace_all(text, dic):
    '''
    This function will replace characters in text given a dictionary of characters to seach for and replace
    '''
    rc = re.compile('|'.join(map(re.escape, dic)))

    def translate(match):
        return dic[match.group(0)]
    return rc.sub(translate, text)


# Dictionary of characters to remove from text
char_replace = {' • ': ''}

# Create list of records read from webscrapper
list_of_rows = []

start_date = "2023-06-01"

# Loop through webpage table, scrap data, and store lists
for i in range(0, 25*50, 25):
    url = 'https://www.prosportstransactions.com/basketball/Search/SearchResults.php?Player=&Team=&BeginDate={}&EndDate=&ILChkBx=yes&InjuriesChkBx=yes&PersonalChkBx=yes&Submit=Search&start={}'.format(start_date,i)
    response = requests.get(url)
    html = response.content

    soup = BeautifulSoup(html, 'lxml')

    table = soup.find('table', attrs={'class': 'datatable center'})

    for row in table.findAll('tr', attrs={'align': 'left'}):
        list_of_cells = []

        for cell in row.findAll('td'):
            text = replace_all(cell.text, char_replace)
            text = text.strip()
            list_of_cells.append(text)
        list_of_rows.append(list_of_cells)

# Store data in a dataframe for manipulation
injuries_df = pd.DataFrame(list_of_rows, columns=[
                           'Date', 'Team', 'Acquired', 'Relinquished', 'Notes'])

acq = injuries_df['Acquired']
rel = injuries_df['Relinquished']

# Remove instances where value is like "Name 1/ Name 2"
injuries_df['Acquired'] = np.where(
    acq.str.contains('/'), acq.str.split('/ ').str[1], acq)
injuries_df['Relinquished'] = np.where(
    rel.str.contains('/'), rel.str.split('/ ').str[1], rel)

# Remove instances where value is like "(some text)"
injuries_df['Acquired'] = injuries_df.Acquired.str.replace(
    r"[\(\[].*?[\)\]]", "")
injuries_df['Relinquished'] = injuries_df.Relinquished.str.replace(
    r"[\(\[].*?[\)\]]", "")

# Write contents to csv file
injuries_df

In [61]:
injuries_df

,Date,Team,Acquired,Relinquished,Notes
0,2023-06-12,Heat,Tyler Herro,,activated from IL
1,2023-06-30,Bucks,,sore calf (DTD),placed on IL with sore calf
2,2023-08-02,76ers,,Montrezl Harrell,torn ACL in knee (out indefinitely)
3,2023-08-05,Nuggets,,Vlatko Cancar,torn ACL in knee (out indefinitely)
4,2023-08-21,Bulls,,Lonzo Ball,recovering from surgery on left knee (out for ...
...,...,...,...,...,...
687,2023-12-15,Pelicans,Zion Williamson,,activated from IL
688,2023-12-15,Rockets,,Jae'Sean Tate,illness (DTD)
689,2023-12-15,Spurs,,Tre Jones,illness (DTD)
690,2023-12-15,Suns,Grayson Allen,,activated from IL


In [179]:
df = injuries_df.copy()
df["Act"]  = (df["Acquired"] != "")
df["DAct"]  = (df["Relinquished"] != "")
df["Player"] = df["Acquired"] + df["Relinquished"]
df["Date"] = pd.to_datetime(df["Date"], format="%Y-%m-%d")
df = df[["Date","Team","Player","Act","DAct","Notes"]]
df = df[df["Player"].str.istitle()].reset_index(drop=True)
df.to_csv('../fdata/NBA_prosptran_injuries_2023.csv', index=False)
df.to_parquet('../fdata/NBA_prosptran_injuries_2023.parquet')

In [180]:
dfp = df.query("Player == 'Nikola Jokic'").reset_index(drop=True)

In [175]:
dfp

,Date,Team,Player,Act,DAct,Notes
0,2023-11-26,Nuggets,Nikola Jokic,False,True,placed on IL with sore lower back
1,2023-11-29,Nuggets,Nikola Jokic,True,False,activated from IL


Did Herro Miss the game on 1st December?

In [173]:
game_date = pd.to_datetime(datetime.date(2023,11,1))
dfp["Comp"] = dfp["Date"] <= game_date
idxi = dfp[dfp["Comp"]].index
if len(idxi) > 0:
    idx = idxi[-1]
    missed_game = dfp["DAct"].loc[idx]
else:
    missed_game = False
missed_game

False